# TP2

In [1]:
import pyspark
from sklearn import preprocessing
import xgboost as xgb
import pandas as pd
import numpy as np

In [2]:
#sc.stop()
sc = pyspark.SparkContext('local[*]')

In [3]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').drop('Unnamed: 0', axis=1)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').drop('Unnamed: 0', axis=1)
postulaciones = pd.read_csv('Sources/postulaciones_for_training.csv').drop('Unnamed: 0', axis=1)

In [4]:
postulaciones = postulaciones.merge(avisos, on='idaviso')
postulaciones = postulaciones.merge(postulantes, on='idpostulante')

In [5]:
#postulaciones['sepostulo']=1

In [6]:
postulaciones.drop(['titulo','descripcion'],axis=1, inplace=True)

In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,fechanacimiento,sexo,nivel
0,1112272884,eOGQRE,1.0,Full-time,Senior / Semi-Senior,Administración,Adecco -Región Office,13,33.0,MASC,11
1,1112274268,eOGQRE,1.0,Full-time,Senior / Semi-Senior,Turismo,BUENAS VIBRAS VIAJES,13,33.0,MASC,11
2,1112358809,eOGQRE,1.0,Full-time,Senior / Semi-Senior,Contabilidad,Eclecsis Recursos Humanos,5,33.0,MASC,11
3,1112288001,eOGQRE,1.0,Full-time,Junior,Administración,CONA CONSULTORES EN RRHH,18,33.0,MASC,11
4,1112352047,eOGQRE,0.0,Full-time,Senior / Semi-Senior,Otros,Suministra,18,33.0,MASC,11


In [8]:
test = pd.read_csv("Sources/test_final_100k.csv")


In [9]:
test = test.merge(avisos, on='idaviso')
test = test.merge(postulantes, on='idpostulante')
test['sepostulo']=1
test.drop(['titulo','descripcion'],axis=1, inplace=True)
test.head()

,id,idaviso,idpostulante,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,fechanacimiento,sexo,nivel,sepostulo
0,0,739260,6M9ZQR,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,42.0,FEM,21,1
1,1,739260,6v1xdL,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,31.0,MASC,9,1
2,2,739260,ezRKm9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,36.0,FEM,12,1
3,3,758580,1Q35ej,Full-time,Otro,Tecnologia / Sistemas,TeleTech,11,69.0,MASC,15,1
4,4,758580,EAN4J6,Full-time,Otro,Tecnologia / Sistemas,TeleTech,11,32.0,FEM,11,1


In [10]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,fechanacimiento,sexo,nivel
0,1112272884,eOGQRE,1.0,Full-time,Senior / Semi-Senior,Administración,Adecco -Región Office,13,33.0,MASC,11
1,1112274268,eOGQRE,1.0,Full-time,Senior / Semi-Senior,Turismo,BUENAS VIBRAS VIAJES,13,33.0,MASC,11
2,1112358809,eOGQRE,1.0,Full-time,Senior / Semi-Senior,Contabilidad,Eclecsis Recursos Humanos,5,33.0,MASC,11
3,1112288001,eOGQRE,1.0,Full-time,Junior,Administración,CONA CONSULTORES EN RRHH,18,33.0,MASC,11
4,1112352047,eOGQRE,0.0,Full-time,Senior / Semi-Senior,Otros,Suministra,18,33.0,MASC,11


In [11]:
test.head()

,id,idaviso,idpostulante,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,fechanacimiento,sexo,nivel,sepostulo
0,0,739260,6M9ZQR,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,42.0,FEM,21,1
1,1,739260,6v1xdL,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,31.0,MASC,9,1
2,2,739260,ezRKm9,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,36.0,FEM,12,1
3,3,758580,1Q35ej,Full-time,Otro,Tecnologia / Sistemas,TeleTech,11,69.0,MASC,15,1
4,4,758580,EAN4J6,Full-time,Otro,Tecnologia / Sistemas,TeleTech,11,32.0,FEM,11,1


In [12]:
resultado = postulaciones['sepostulo']
resultadob = test['sepostulo']
postulaciones.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop(['sepostulo','idaviso','idpostulante'],axis=1,inplace=True)
test.drop('id',axis=1,inplace=True)

In [13]:
test = pd.concat([test,postulaciones])

In [14]:
test.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,cluster,fechanacimiento,sexo,nivel
0,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,42.0,FEM,21
1,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,31.0,MASC,9
2,Full-time,Jefe / Supervisor / Responsable,Comercial,BUMERAN.COM ARGENTINA,17,36.0,FEM,12
3,Full-time,Otro,Tecnologia / Sistemas,TeleTech,11,69.0,MASC,15
4,Full-time,Otro,Tecnologia / Sistemas,TeleTech,11,32.0,FEM,11


In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in test.columns:
    if not postulaciones[col].dtype == 'O': continue
    #postulaciones[col] = postulaciones.apply(lambda x: str(x[col]),axis=1)
    test[col] = test[col].fillna('na')
    le.fit(test[col])
    test[col] = le.transform(test[col])
    #postulaciones[col] = le.transform(postulaciones[col])

In [16]:
#postulaciones.rename(columns={x:int(y) for x,y in zip(postulaciones.columns,range(1,len(postulaciones.columns)+1))},inplace=True)
test.rename(columns={x:int(y) for x,y in zip(test.columns,range(1,len(test.columns)+1))},inplace=True)

In [17]:
postulaciones = test[100000:]
test = test[:100000]

In [18]:
postulaciones.head()

,1,2,3,4,5,6,7,8
0,1,4,2,212,13,33.0,2,11
1,1,4,182,452,13,33.0,2,11
2,1,4,42,1339,5,33.0,2,11
3,1,2,2,718,18,33.0,2,11
4,1,4,137,3704,18,33.0,2,11


In [19]:
test.head()

,1,2,3,4,5,6,7,8
0,1,1,30,458,17,42.0,1,21
1,1,1,30,458,17,31.0,2,9
2,1,1,30,458,17,36.0,1,12
3,1,3,170,3860,11,69.0,2,15
4,1,3,170,3860,11,32.0,1,11


In [20]:
aviso = None
postulantes = None
import xgboost as xgb
# read in data
dtrain = xgb.DMatrix(data=postulaciones,label=resultado)
dtest = xgb.DMatrix(data=test, label=resultadob)
# specify parameters via map
param = {'max_depth':50, 'eta':1, 'silent':1, 'objective':'multi:softmax','num_class':3, 'subsample':1}
num_round = 50

bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

In [21]:
preds[preds==1]

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [ ]:
test = pd.read_csv("Sources/test_final_100k.csv")



In [ ]:
test['sepostulo'] = preds

In [ ]:
test.head()

In [ ]:
test.drop(['idaviso','idpostulante'],axis=1,inplace=True)

In [ ]:
test.to_csv('Sources/resultados.csv', index=False)

In [ ]:
pd.concat([test['idaviso'],postulaciones['idaviso']])